##### Forked from the Tensorflow's [official colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb) on Image Classification.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image classification

This tutorial shows how to classify cats or dogs from images. It builds an image classifier using a `tf.keras.Sequential` model and also the functional API. You will get some practical experience and develop intuition for the following concepts:

* Building _data input pipelines_ to efficiently work with data on disk to use with the model.
* _Overfitting_ —How to identify and prevent it.
* _Data augmentation_ and _dropout_ —Key techniques to fight overfitting in computer vision tasks to incorporate into the data pipeline and image classifier model.

This tutorial follows a basic machine learning workflow:

1. Examine and understand data
2. Build an input pipeline
3. Build the model
4. Train the model
5. Test the model
6. Improve the model and repeat the process

## Import packages

Let's start by importing the required packages. The `os` package is used to read files and directory structure, NumPy is used to convert python list to numpy array and to perform required matrix operations and `matplotlib.pyplot` to plot the graph and display images in the training and validation data.

Import Tensorflow and the Keras classes needed to construct our model.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input,
    Dense,
    Conv2D,
    Flatten,
    Dropout,
    MaxPooling2D,
    BatchNormalization,
    Add,
    GlobalAveragePooling2D,
    MaxPool2D,
    ReLU
)
from tensorflow.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping,
    ModelCheckpoint
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import tensorflow_datasets as tfds
import os
import numpy as np
import matplotlib.pyplot as plt

## Load data

Let us load the Cats vs Dogs dataset from `tfds`. We shuffle and use the first 90\% for training and the rest for validation. The dataset contains around 20k training images of cats and dogs.

In [ ]:
train_ds, val_ds =  tfds.load('cats_vs_dogs', as_supervised=True, shuffle_files=True, split=['train[:90%]', 'train[90%:]'])

For convenience, set up variables to use while pre-processing the dataset and training the network.

In [ ]:
batch_size = 128
epochs = 15
IMG_HEIGHT = 180
IMG_WIDTH = 180

## Data preparation

Format the images into appropriately pre-processed floating point tensors before feeding to the network:

1. Decode contents of these images and convert it into proper grid format as per their RGB content.
2. Convert them into floating point tensors.
3. Rescale the tensors from values between 0 and 255 to values between 0 and 1, as neural networks prefer to deal with small input values.

In [ ]:
def convert(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH]) # Resize image
    return image, label

train_batches = (
    train_ds
    .shuffle(512)
    .map(convert, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(batch_size)
)

validation_batches = (
    val_ds
    .map(convert, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(batch_size)
)

### Visualize training images

Visualize the training images by extracting a batch of images from the training generator—which is 32 images in this example—then plot five of them with `matplotlib`.

In [ ]:
sample_ds = list(train_ds.take(5).as_numpy_iterator())
sample_imgs, _ = zip(*sample_ds)

The `next` function returns a batch from the dataset. The return value of `next` function is in form of `(x_train, y_train)` where x_train is training features and y_train, its labels. Discard the labels to only visualize the training images.

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_imgs)

## Create the model

The model consists of three convolution blocks with a max pool layer in each of them. There's a fully connected layer with 512 units on top of it that is activated by a `relu` activation function.

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

### Compile the model

For this tutorial, choose the *ADAM* optimizer and *binary cross entropy* loss function. To view training and validation accuracy for each training epoch, pass the `metrics` argument.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Model summary

View all the layers of the network using the model's `summary` method:

In [ ]:
model.summary()

### Train the model

Use the `fit` method to train the network.

In [ ]:
history = model.fit(
    train_batches, epochs=epochs, validation_data=validation_batches
)

### Visualize training results

Now visualize the results after training the network.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

As you can see from the plots, training accuracy and validation accuracy are off by large margin and the model has achieved only around **75-80%** accuracy on the validation set.

Let's look at what went wrong and try to increase overall performance of the model.

## Overfitting

In the plots above, the training accuracy is increasing linearly over time, whereas validation accuracy stalls around 75-80% in the training process. Also, the difference in accuracy between training and validation accuracy is noticeable—a sign of *overfitting*.

When there are a small number of training examples, the model sometimes learns from noises or unwanted details from training examples—to an extent that it negatively impacts the performance of the model on new examples. This phenomenon is known as overfitting. It means that the model will have a difficult time generalizing on a new dataset.

There are multiple ways to fight overfitting in the training process. In this tutorial, you'll use *data augmentation* and add *dropout* to our model.

## Data augmentation

Overfitting generally occurs when there are a small number of training examples. One way to fix this problem is to augment the dataset so that it has a sufficient number of training examples. Data augmentation takes the approach of generating more training data from existing training samples by augmenting the samples using random transformations that yield believable-looking images. The goal is the model will never see the exact same picture twice during training. This helps expose the model to more aspects of the data and generalize better.

### Augment and visualize data

Apply the following augmentation steps: (1) random horizontal flip, (2) random rotation.

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images[tf.newaxis, ...], training=True)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype('uint8'))
    plt.axis('off')

## Dropout

Another technique to reduce overfitting is to introduce *dropout* to the network. It is a form of *regularization* that forces the weights in the network to take only small values, which makes the distribution of weight values more regular and the network can reduce overfitting on small training examples. Dropout is one of the regularization technique used in this tutorial

When you apply dropout to a layer it randomly drops out (set to zero) number of output units from the applied layer during the training process. Dropout takes a fractional number as its input value, in the form such as 0.1, 0.2, 0.4, etc. This means dropping out 10%, 20% or 40% of the output units randomly from the applied layer.

When appling 0.1 dropout to a certain layer, it randomly kills 10% of the output units in each training epoch.

Create a network architecture with this new dropout feature and apply it to different convolutions and fully-connected layers.

## Creating a new network with Dropouts

Here, you apply dropout to first and last max pool layers. Applying dropout will randomly set 20% of the neurons to zero during each training epoch. This helps to avoid overfitting on the training dataset. We also use the functional API to build this model.

In [ ]:
x = inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH ,3))
x = data_augmentation(x, training=True)
x = Conv2D(16, 3, padding='same', activation='relu',
           input_shape=(IMG_HEIGHT, IMG_WIDTH ,3))(x)
x = MaxPooling2D()(x)
x = Dropout(0.2)(x)
x = Conv2D(32, 3, padding='same', activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = MaxPooling2D()(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
output = Dense(1)(x)
model_new = tf.keras.Model(inputs, output, name="dropout_model")

### Compile the model

After introducing dropouts to the network, compile the model and view the layers summary.

In [ ]:
model_new.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

model_new.summary()

### Train the model

After successfully introducing data augmentations to the training examples and adding dropouts to the network, train this new network:

In [ ]:
history = model_new.fit(
    train_batches, epochs=epochs, validation_data=validation_batches
)

### Visualize the model

Visualize the new model after training, you can see that there is significantly less overfitting than before. The accuracy should go up after training the model for more epochs.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## TODO: Create a ResNet model

Implement ResNet-18 using Tensorflow's [functional API](https://www.tensorflow.org/guide/keras/functional). Fill out the functions `conv`, `residual`, and `block`. Then use them to create the ResNet model in the function `resnet`.

Link to ResNet paper: [https://arxiv.org/pdf/1512.03385.pdf](https://arxiv.org/pdf/1512.03385.pdf)

In [ ]:
def conv(x, filters, size, strides=1):
    """
    Apply conv, batch norm, and activation to input x. Use strides > 1 to
    downsample the output.
    Relevant documentation:
      - Conv2D: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
      - BatchNorm: https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization
      - ReLU: https://www.tensorflow.org/api_docs/python/tf/keras/layers/ReLU
    Example:
      x = conv(x)
      x = batchnorm(x)
      x = activation(x)
      return x
    """

    ### YOUR CODE GOES HERE ###

    pass

def residual(x, filters, size):
    """
    Create a simple residual block as defined in the paper.
    Example:
      prev = x
      x = resnet_conv(x, filters, size)
      x = resnet_conv(x, filters, size)
      x = Add()([prev, x])
      return x
    """

    ### YOUR CODE GOES HERE ###

    pass

def block(x, filters, size, num_blocks):
    """
    Create a series of residuals of length num_blocks. This should
    constitute the colored chain of residuals shown in Figure 3 of the
    ResNet paper (https://arxiv.org/pdf/1512.03385.pdf)
    Example:
      x = conv(x, filters, size, 2)
      for _ in range(num_blocks):
        x = residual(x, filters, size)
      return x
    """

    ### YOUR CODE GOES HERE ###

    pass

def resnet(input_shape, name="resnet"):
    """
    Create the final ResNet architecture using the modular API. Make use of the
    functions defined above (conv, residual, block). Try to implement the
    18-layer ResNet architecture defined in Table 1 of the ResNet paper.
    """
    inputs = Input(shape=input_shape)

    ### YOUR CODE GOES HERE ###

    outputs = Dense(1, activation="sigmoid")(x)
    return tf.keras.Model(inputs, outputs, name=name)

### Compile the model

In [ ]:
model_resnet = resnet(input_shape=[IMG_WIDTH, IMG_HEIGHT, 3])

model_resnet.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

model_resnet.summary()

### Train the model

In [ ]:
callbacks = [
            ReduceLROnPlateau(verbose=1),
            EarlyStopping(patience=5, verbose=1),
            ModelCheckpoint('resnet_train_{epoch}.tf',
                            verbose=1, save_weights_only=True)
            ]

# model will stop training if validation accuracy keeps falling for 5 consecutive epochs
history = model_resnet.fit(
    train_batches, epochs=epochs, validation_data=validation_batches, callbacks=callbacks
)

### Visualize the model

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Test your model on a different dataset

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

test_dir = os.path.join(PATH, 'validation')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

num_cats_test = len(os.listdir(test_cats_dir))
num_dogs_test = len(os.listdir(test_dogs_dir))

print('total test cat images:', num_cats_test)
print('total test dog images:', num_dogs_test)

test_image_generator = ImageDataGenerator(rescale=1./255)
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                         directory=test_dir,
                                                         target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                         class_mode='binary')

# Change this to choose specific checkpoint weights (which gives highest validation accuracy)
ckpt = "resnet_train_15.tf"
model_resnet.load_weights(ckpt)

stats = model_resnet.evaluate(
    test_data_gen, batch_size=batch_size
)
print("test loss, test accuracy:", stats)